# Preparación de datos

In [1]:
#nltk.download('punkt')
#datos para la tokenización de nltk
#nltk.download('punkt_tab')


#nltk.download('stopwords')
#stop words reconocidos en nltk

In [ ]:
pip 

In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from collections import Counter

#para la puntuación
from collections import Counter
from itertools import chain
import string

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)


ModuleNotFoundError: No module named 'nltk'

# División de muestras en entrenamiento (train) y prueba (test)

In [1]:


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)
print(len(X_train))

NameError: name 'train_test_split' is not defined

### Pregunta 1

Ajustar los datos de entrenamiento `X_train` utilizando un `count_vectorizer` con parámetros predeterminados.

¿Cuál es el token más largo en el vocabulario?

*Esta función debería devolver una cadena.*

In [4]:


def respuesta_uno (X_train):
    #asignamos modelo
    vectorizer = CountVectorizer()
    #ajustamos el modelo
    vectorizer.fit(X_train)  
    #Obtenemos los tokens
    vocab = vectorizer.get_feature_names_out()  
    #obtenemos el token mas largo
    return max(vocab, key=len)
    
respuesta_uno(X_train)

'com1win150ppmx3age16subscription'

### Pregunta 2

¿Cuál es el número promedio de caracteres por documento para los documentos no spam y spam?

*Esta función debe devolver una tupla (promedio de # caracteres no es spam, promedio # caracteres spam).*

In [5]:
def respuesta_dos():
    #agragamos columna con tamaño
    spam_data['length'] =spam_data["text"].apply(len)
    #hacemos tabla dinamica
    tabla_dinamica = spam_data.pivot_table(values="length", index="target", aggfunc="mean")

    return tabla_dinamica["length"][0],tabla_dinamica["length"][1] 
respuesta_dos()

(71.02362694300518, 138.8661311914324)

### Pregunta 3

¿Cuál es el número promedio de dígitos por documento para los documentos no spam y spam?

*Esta función debe devolver una tupla (promedio de # dígitos no es spam, promedio # dígitos spam).*

In [6]:
def respuesta_tres():
    
    spam_data["digitos"] = spam_data["text"].apply(lambda x: re.findall(r'\d+', x))

    spam_data["digitos"] = spam_data["digitos"].apply(lambda x: ''.join(x))

    spam_data['len_digitos'] =spam_data["digitos"].apply(len)
    #hacemos tabla dinamica
    tabla_dinamica = spam_data.pivot_table(values="len_digitos", index="target", aggfunc="mean")
    
    return tabla_dinamica["len_digitos"][0],tabla_dinamica["len_digitos"][1] 
respuesta_tres()

(0.2992746113989637, 15.759036144578314)

### Pregunta 4

¿Cuál es el número promedio de caracteres que no son palabras (cualquier cosa que no sea una letra, un dígito o un guión bajo) por documento para los documentos que no son spam y spam?

*Sugerencia: utilice las clases de caracteres `\ w` y` \ W`*

*Esta función debe devolver una tupla (promedio de # caracteres que no son palabras, no spam, promedio de # caracteres que no son palabras, spam).*

In [7]:
def pregunta_cuatro():
    
    spam_data["no_w"] = spam_data["text"].apply(lambda x: re.findall(r'\W', x))


    spam_data['len_no_w'] =spam_data["no_w"].apply(len)
    #hacemos tabla dinamica
    tabla_dinamica = spam_data.pivot_table(values="len_no_w", index="target", aggfunc="mean")
    
    return tabla_dinamica["len_no_w"][0],tabla_dinamica["len_no_w"][1] 
pregunta_cuatro()

(17.29181347150259, 29.041499330655956)

### Pregunta 5

¿Cuál es el tamaño del vocabulario en `X_train` y `X_test`, primero utilizando la función `fit_transform` en ambos (train y test), luego utilizando `fit_transform` sobre el train y solo `transform` en el test



*Esta función debe devolver dos tuplas una con `fit_transform` y la otra con `transform` (vocabulario en `X_train`, vocabulario en `X_test`), (vocabulario en `X_train`, vocabulario en `X_test`).*.*

Cuando aplicamos repetidamente `fit_transform` lo que hace es resetear el modelo. Por lo que para aplicar `fit_transform` en train como en test lo que haremos sera juntar los textos o tomar el texto original.

In [8]:
def respuesta_cinco():
    
    vectorizer_p5 = CountVectorizer()
    X_train_transformed_1 = vectorizer_p5.fit_transform(X_train)
    X_test_transformed_1 = vectorizer_p5.fit_transform(X_test)
    #aca se reinstancia el vectorizador por loque pierde sentido el haber aplicado el fit sobre train
    vocab_train_1 = vectorizer_p5.transform(X_test)

    vectorizer_p5_2 = CountVectorizer()
    X_train_transformed_2 = vectorizer_p5_2.fit_transform(X_train)
    vocab_train_2 = vectorizer_p5_2.transform(X_test)

    return vocab_train_1.sum(),vocab_train_2.sum()   

respuesta_cinco()

(20403, 18942)

### Pregunta 6

¿Cuales son las 10 palabras mas frecuentes (sin tener en cuenta *Stopwords*) en los documentos que no son spam y spam?


*Esta función debe devolver una tupla (palabras mas frecuentes, no spam, palabras mas frecuentes, spam).*

In [52]:
def pregunta_seis():
    stop_words = set(stopwords.words('english'))
    spam_data["text_no_sw"] = spam_data["text"].apply(
    lambda x: [word for word in word_tokenize(x) if word.lower() not in stop_words]
    )
    spam_data["text_sw"] = spam_data["text"].apply(
    lambda x: [word for word in word_tokenize(x) if word.lower() in stop_words]
    )


    # Separar el DataFrame en dos: uno para "spam" y otro para "ham"
    df_0 = spam_data[spam_data["target"] == 0]
    df_1 = spam_data[spam_data["target"] == 1]
    
    todas_las_palabras = list(chain.from_iterable(df_0["text_no_sw"]))

    todas_las_palabras_limpias = [
        token.lower() 
        for token in todas_las_palabras 
        if token.isalpha() and token not in string.punctuation
    ]

    frecuencia = Counter(todas_las_palabras_limpias)
    top_10 = frecuencia.most_common(10)

    todas_las_palabras_1 = list(chain.from_iterable(df_1["text_no_sw"]))

    todas_las_palabras_limpias = [
        token.lower() 
        for token in todas_las_palabras_1
        if token.isalpha() and token not in string.punctuation
    ]

    frecuencia_1 = Counter(todas_las_palabras_limpias)
    top_10_1 = frecuencia_1.most_common(10)

    return top_10,top_10_1
pregunta_seis()

([('u', 989),
  ('gt', 318),
  ('lt', 316),
  ('get', 301),
  ('ok', 261),
  ('go', 248),
  ('got', 242),
  ('ur', 237),
  ('know', 236),
  ('like', 231)],
 [('call', 346),
  ('free', 219),
  ('txt', 156),
  ('u', 144),
  ('ur', 144),
  ('mobile', 123),
  ('text', 121),
  ('stop', 114),
  ('claim', 113),
  ('reply', 104)])

### Pregunta 7

¿Cuales son las 10 palabras mas frecuentes (solo teniendo en cuenta *Stopwords*) en los documentos que no son spam y spam?


*Esta función debe devolver una tupla (palabras mas frecuentes, no spam, palabras mas frecuentes, spam).*

In [53]:
def pregunta_siete():
    stop_words = set(stopwords.words('english'))
    spam_data["text_no_sw"] = spam_data["text"].apply(
    lambda x: [word for word in word_tokenize(x) if word.lower() not in stop_words]
    )
    spam_data["text_sw"] = spam_data["text"].apply(
    lambda x: [word for word in word_tokenize(x) if word.lower() in stop_words]
    )


    # Separar el DataFrame en dos: uno para "spam" y otro para "ham"
    df_0 = spam_data[spam_data["target"] == 0]
    df_1 = spam_data[spam_data["target"] == 1]
    
    todas_las_palabras = list(chain.from_iterable(df_0["text_sw"]))

    todas_las_palabras_limpias = [
        token.lower() 
        for token in todas_las_palabras 
        if token.isalpha() and token not in string.punctuation
    ]

    frecuencia = Counter(todas_las_palabras_limpias)
    top_10 = frecuencia.most_common(10)

    todas_las_palabras_1 = list(chain.from_iterable(df_1["text_sw"]))

    todas_las_palabras_limpias = [
        token.lower() 
        for token in todas_las_palabras_1
        if token.isalpha() and token not in string.punctuation
    ]

    frecuencia_1 = Counter(todas_las_palabras_limpias)
    top_10_1 = frecuencia_1.most_common(10)

    return top_10,top_10_1
pregunta_siete()

([('i', 2851),
  ('you', 1932),
  ('to', 1554),
  ('the', 1118),
  ('a', 1048),
  ('and', 855),
  ('in', 814),
  ('me', 774),
  ('my', 747),
  ('is', 740)],
 [('to', 687),
  ('a', 376),
  ('you', 296),
  ('your', 263),
  ('the', 206),
  ('for', 202),
  ('now', 199),
  ('or', 188),
  ('is', 158),
  ('on', 144)])